## Setup paquetes y Google Drive

In [79]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import os
import time

## Descarga y preprocesamiento de datos

Hay cambios del notebook de partida (de encoding `utf-8` a `latin-1`).

In [80]:
texto = open("3_textos_literatura_española/Lazarillo-De-Tormes.txt", 'rb').read().decode(encoding='utf-8')
print('Longitud del texto:        {} carácteres'.format(len(texto)))

vocab = sorted(set(texto))

print ('El texto está compuesto de estos {} carácteres:'.format(len(vocab)))
print (vocab)

Longitud del texto:        107332 carácteres
El texto está compuesto de estos 80 carácteres:
['\n', '\r', ' ', '!', '"', '(', ')', ',', '-', '.', '1', '4', '5', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '{', '}', '¡', '«', '»', '¿', 'É', 'á', 'é', 'ê', 'í', 'ñ', 'ó', 'ú', 'ü']


### Procesamiento de los textos
#### Mapeo de caracteres

In [81]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for char,_ in zip(char2idx, range(len(vocab))):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '!' :   3,
  '"' :   4,
  '(' :   5,
  ')' :   6,
  ',' :   7,
  '-' :   8,
  '.' :   9,
  '1' :  10,
  '4' :  11,
  '5' :  12,
  ':' :  13,
  ';' :  14,
  '?' :  15,
  'A' :  16,
  'B' :  17,
  'C' :  18,
  'D' :  19,
  'E' :  20,
  'F' :  21,
  'G' :  22,
  'H' :  23,
  'I' :  24,
  'J' :  25,
  'L' :  26,
  'M' :  27,
  'N' :  28,
  'O' :  29,
  'P' :  30,
  'Q' :  31,
  'R' :  32,
  'S' :  33,
  'T' :  34,
  'U' :  35,
  'V' :  36,
  'Y' :  37,
  'Z' :  38,
  '_' :  39,
  'a' :  40,
  'b' :  41,
  'c' :  42,
  'd' :  43,
  'e' :  44,
  'f' :  45,
  'g' :  46,
  'h' :  47,
  'i' :  48,
  'j' :  49,
  'k' :  50,
  'l' :  51,
  'm' :  52,
  'n' :  53,
  'o' :  54,
  'p' :  55,
  'q' :  56,
  'r' :  57,
  's' :  58,
  't' :  59,
  'u' :  60,
  'v' :  61,
  'x' :  62,
  'y' :  63,
  'z' :  64,
  '{' :  65,
  '}' :  66,
  '¡' :  67,
  '«' :  68,
  '»' :  69,
  '¿' :  70,
  'É' :  71,
  'á' :  72,
  'é' :  73,
  'ê' :  74,
  'í' :  75,
  'ñ' :  76,

Pasamos cada texto a un array de enteros

In [82]:
text_as_int = np.array([char2idx[c] for c in texto])

print ('texto : {}'.format(repr(texto[:50])))
print ('{}'.format(repr(texto[:50])))

texto : 'LA VIDA DE LAZARILLO DE TORMES Y DE SUS FORTUNAS Y'
'LA VIDA DE LAZARILLO DE TORMES Y DE SUS FORTUNAS Y'


### Preparación de los datos para entrenar la RNN

Para entrenar el modelo creamos un conjunto de datos con el contenido de text_as_init. Para ello utilizamos la función tf.data.Dataset.from_tensor_slices.
A este conjunto de datos lo dividiremos en secuencias de seq_length+1 al aplicar el método batch()


In [83]:
# Creamos una función `split_input_target` que devolverá el conjunto de datos
# de entrenamiento (los datos de entrada como los datos de salida)
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

#Agrupamos los dataset en batches de 64 .
# Así tendriamos los datos de entrenamiento con batches compuestos de 64 parejas
# de secuencias de 100 integers de 64 bits
BATCH_SIZE = 64
BUFFER_SIZE = 10000

In [84]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

#Aplicamos split_input_target a todas las secuencias utilizando el método map()
dataset = sequences.map(split_input_target)

print("\n\n")

#Los dataset contienen un conjunto de parejas (100 caracteres del texto original, la correspondiente salida ). Vamos a mostrar la primera pareja.
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

  print(dataset)

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)

'LA VIDA DE LAZARILLO DE TORMES Y DE SUS FORTUNAS Y ADVERSIDADES\r\nAutor desconocido.\r\nEdición de Burgo'
's, 1554.\r\n{Interpolaciones de la edición de Alcalá}\r\n_cursiva_\r\n\r\n\r\n\r\n\r\nPrólogo\r\n\r\n\r\n\r\nYo por bien te'
'ngo que cosas tan señaladas, y por ventura nunca oídas ni vistas, vengan a noticia de muchos y no se '
'entierren en la sepultura del olvido, pues podría ser que alguno que las lea halle algo que le agrade'
', y a los que no ahondaren tanto los deleite; y a este propósito dice Plinio que no hay libro, por ma'
'lo que sea, que no tenga alguna cosa buena; mayormente que los gustos no son todos unos, mas lo que u'
'no no come, otro se pierde por ello. Y así vemos cosas tenidas en poco de algunos, que de otros no lo'
' son. Y esto, para ninguna cosa se debría romper ni echar a mal, si muy detestable no fuese, sino que'
' a todos se comunicase, mayormente siendo sin perjuicio y pudiendo sacar della algún fruto; porque si'
' así no fuese, muy pocos escribirían 

### Construcción del modelo RNN

In [85]:
#Crearemos una función que cree un modelo RNN con tres capas
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, BatchNormalization

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = Sequential()
  #Añadimos la capa de tipo word embedding
  model.add(Embedding(input_dim=vocab_size,
                      output_dim=embedding_dim,
                      #batch_input_shape=[batch_size, None] Deprecated
                      ))
  #Añadimos la capa de tipo LSTM
  model.add(LSTM(rnn_units,
                 return_sequences=True,
                 stateful=True,
                 recurrent_initializer='glorot_uniform'))
  model.add(Dense(512, activation="relu"))
  model.add(BatchNormalization())
  model.add(Dropout(0.4))


  #Añadimos la capa de tipo Dense
  model.add(Dense(vocab_size))
  return model

In [86]:
embedding_dim = 256
rnn_units = 1024

In [87]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [88]:
for input_example_batch, target_example_batch in dataset.take(1):
  print("Input:", input_example_batch.shape, "# (batch_size, sequence_length)")
  print("Target:", target_example_batch.shape, "# (batch_size, sequence_length)")

Input: (64, 100) # (batch_size, sequence_length)
Target: (64, 100) # (batch_size, sequence_length)


In [89]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print("Prediction : ", example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

Prediction :  (64, 100, 80) # (batch_size, sequence_length, vocab_size)


2025-06-24 21:28:10.595680: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [90]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices_characters = tf.squeeze(sampled_indices,axis=-1).numpy()

In [91]:
print(sampled_indices_characters)

[61 42 26 77 25 12 55 76 45 64 26 37 65 41 67 24 55 64 50 79  3 58 51 61
 53 59 29  0  7 64 41  1 20 57 46 33 61 10 17 26 45 56 77 69 25 43 45 43
 23 59 20 78 55 77 53 36  0 72 72 30 77  6 57 47 18 76 79 51 58 39 27 59
 37 13 70 20 75 15 77 23 74 14  2 29 50 47 10 47 45 15 49  7 25 64 34 39
  8  1 41 16]


### Entrenamiento del modelo RNN

In [92]:
#Creamos la función de perdida, usaremos el categorical pues estamos considerando datos categóricos
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [93]:
#Compilamos el modelo
model.compile(optimizer='adam', loss=loss)

In [94]:
#configuramos los checkpoints

checkpoint_dir = './training_checkpoints_Lazarillo'

# nombre fichero
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [95]:
# Implementación EarlyStopping
from tensorflow.keras.callbacks import EarlyStopping

early_stopping_callback = EarlyStopping(
    monitor='loss',
    patience=10,
    min_delta=0.01,
    restore_best_weights=True
)

In [96]:
#Entrenamos el modelo
EPOCHS=100
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback, early_stopping_callback])

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 601ms/step - loss: 3.6581
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 596ms/step - loss: 2.4085
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 624ms/step - loss: 2.1887
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 635ms/step - loss: 2.0727
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 613ms/step - loss: 2.0008
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 619ms/step - loss: 1.9467
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 619ms/step - loss: 1.8978
Epoch 8/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 626ms/step - loss: 1.8619
Epoch 9/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 645ms/step - loss: 1.8241
Epoch 10/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 646ms/step - loss: 1.7929
Epoch 11/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 664ms/step - loss: 1.7587
Epoch 12/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 631ms/step - loss: 1.7235
Epoch 13/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 626ms/step - loss: 1.6817
Epoch 14/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 10s 641ms/step - loss: 1.6633
Epoch 15/100
16/16 ━━━━━━━━━━

In [97]:
model.save("model_lazarillo_100_2025.keras")

In [98]:
from keras.models import load_model
from keras import losses # Import the losses module

# Assuming your original loss function was, for example, binary_crossentropy
loaded_model = load_model("model_lazarillo_100_2025.keras",
                          custom_objects={'loss': losses.sparse_categorical_crossentropy})
# or if it was a custom loss function
# loaded_model = load_model("model_paquita_100_2024.keras", custom_objects={'loss': my_custom_loss_function})

In [99]:
model = build_model(len(vocab), embedding_dim, rnn_units, batch_size=1)
input_shape = (1, 100)  # Replace 100 with your actual sequence length
model.build(input_shape=input_shape) # Or model.build(tf.TensorShape([1, None]))

model.load_weights("model_lazarillo_100_2025.keras")

In [106]:
#Creamos una función generar_texto que generará texto a partir de una palabra de partida
def generate_text(model, start_string):

  num_generate = 1000
  input_eval = [char2idx[s] for s in start_string]

  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []


  # temperature = 0.7 # cuanto mas alto el numero, mas directa la salida de los logits
  temperature = 0.3

#  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)

      predictions = tf.squeeze(predictions, 0)

      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()


      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [108]:
print(generate_text(model, start_string=u"Yo, señor, aunque soy hijo de padres humildes"))

# print(generate_text(model, start_string=u"Cuando llegamos al mesón,"))

Yo, señor, aunque soy hijo de padres humildese que amos tú esta hora entró una vieja que ensalmaba, y los vecinos, y comiénzanme a quitar trapos de la cabeza y viejo al del con el cabo de la capa sobre el lado izquierdo, sacó una llave de la manga y abrió su puerta a los que en ella entraban, aunque dentro della estaba un patio pequeño y razonables cámaras.

Desque tan recio como si diera con una gran calabaza, y cayó luego para atrás, así con ella, había muerto en la de los Gelves, y que ella confiaba en Dios no saldría peor hombre que desea mucho el provecho de las ánimas; mas pregunten a su merced si le pesa cuando le dicen al altoración y decir:

"¿Qué diremos a esto? ¡Nunca haber sentido ratones en esta casa sino agor de la Sagra de Toledo había predicado dos o tres días, haciendo sus acostumbradas diligencias, y decía:

"¿Qué tienes este había tenido a buscar clavos por la casa y por las paredes y tablillas a atapárselos. Venida la noche y su reposo, y también porque consideren l

# Intentos básicos
## Intento 1.1: 300 épocas y 0.4 de temperatura
```
Yo, señor, aunque soy hijo de padres humildes alegres del arcaz, sentí que mi amo dormía, porque lo mostraba con roncar y en unos resoplidos grandes bien aquel día, maldito el gusto yo tomaba en ello, ni en aquellos tres días torné en mi color; y me demediar dende en adelante la triste vida. Y así estuve con ello aquel día y otro gozoso. Mas no estaba prestada una ratonera, y con cortezas de queso que a los vecinos pedía, contino el gato estaba armados la sepultura. Pues si deste desisto y doy en otro más bajo, ¿qué será sino fenecer?"

Con esto no por No mires a aquél que no sabe lo que hace ni dice; mas la injuria a ti hecha, te suplico, y por justicia y declarar a voces sus delitos: pregonero, hablando en buen romance, en el cual oficia a ayudar y a defender la entrada. El cual algo alterado, pensando que fuese otra cosa, me dijo:

"Lázaro, llega el oído a este toro, y oirás gran ruido dentro dél."

Yo simpla lumbre. Y, ya que hubo acabado la misa y echada la bendición, tomóla con un pañizuelo, bien envuelta la contenta
```
### Observaciones:
Es más coherente que el modelo de la Celestina, aunque si muestra posibilidad de overfitting, porque hay frases completas que las saca directo del texto. Habiendo ya mejorado el modelo de la Celestina, aplico ya directamente dichas mejoras.

# Intentos con cambios
### EarlyStopping
Agregamos `EarlyStopping` por lo frecuente que es la repetición en el modelo de los intentos anteriores. Siguiendo con 300 épocas, el entrenamiento ahora frena temprano. Por ejemplo, en la primera ejecución con 300 épocas frena en la época 89 con una pérdida de 0.2089.
```
Epoch 89/300
58/58 ━━━━━━━━━━━━━━━━━━━━ 32s 552ms/step - loss: 0.2089
```
Esto indica que el modelo había dejado de mejorar significativamente y que continuar entrenando hubiera sido ineficiente y contraproducente por overfitting.
### BatchNormalization
Agregamos `BatchNormalization` porque estabiliza y acelera el entrenamiento al normalizar la salida de la capa `Dense`. Esto mejora la propagación del gradiente, reduce la sensibilidad a la inicialización de pesos y regulariza el modelo. También permite que la red entrene con mayor estabilidad al combinarse con activaciones como `ReLU.`
### Dropout
Bajamos el dropout porque el valor original (0.5) era demasiado agresivo y podía estar dificultando el aprendizaje de patrones. Al reducirlo a 0.3, se mantiene cierta regularización para prevenir el sobreajuste.

## Intento 1.1: 300 épocas y 0.5 temperatura
```
Yo, señor, aunque soy hijo de padres humildes, mas no lo hice sin no me lo tuviera más altos, como yo, no les han de hablar menos de: «Beso las manos de vuestra merced», o por lo menos las cosas de la honra, en que el día de hoy está todo el caudal de los hombres de bien. Pues te ha de mantenerto, pienso que me sintió, y dende en adelante mudó propósito, y asentaba su jarro entre las piernas, teniendo con el fino muy a tendido des veces, hablando con reverencia de V.M., porque está ella delante."

Entonces mi mujer echó juramente las pajas do yo estaba en el púlpito de rodillas, las manos y los ojos puestos en el cielo, transportado en la de aquella marerdad puedo decir nacido en el río. Pues siendo yo niño de ocho años, achacaron a mi padre ciertas sabido como pensando con ellos parcial, y cuánto más hicieron los que, siéndoles contraria, con fuerza y maña remando, me parecía que hacía sinjusticia en no se las reír.

Y en cuanto esto pasaba, a la memoria me vinieron a reñir y a haber malas palabras. Él llamó al alguacil ladr
```
### Observaciones
Repite mucho texto literal del original, incluso frases largas enteras. Aunque la temperatura es alta, no genera nada especialmente creativo ni distinto, solo mezcla mal partes ya vistas. Probé con otras temperaturas y es igual: copia y pega.

## Intento 1.2: 50 épocas y 0.2
```
Yo, señor, aunque soy hijo de padres humildes, y hijos malas y partendose en camino y manera provechosa; y con favor que tuve de amigos y señores, y allí algunas malas con el deseo, en tanto que el ciego sacaba de la bolsa el dinero, saqué la longaniza y muy presto me vino a tan buena recia, y aunque mozo por mejor decir, morí. De la taberna nunca le traje una blanca de vino, solor mi amo visto el daño así del pan como del agujero que yo había hecho, y comenzó a dar a los diaba de haber llevado muy buenas lanzas. ¿Qué hiciera que en la calle andaba con esto acos mis pajas, y para todos sus de las cuales él tando con el diese el de antes una caba de los amos que yo hallaba y había por bien de que ella entrase y saliese, de noche y de día, pues estaba biendo y decirle en ella cuerta de la senan de otras tantas perdeciose en falsar con las cuales daba y hallé en el cuel de la bula tomaban. Visto por el asunto de mi amo lo que pasaba y mus cantenos y en el ciego, la tenía tan hecha bolsa que me acaeció tener en el culcil diente de su
```
### Observaciones
Aunque entrenado menos, sigue repitiendo mucho. Con temperatura baja se vuelve todavía más pegado al corpus original, y muchas frases están cortadas o no llegan a ningún lado. No hay mucha invención ni combinación original, simplemente junta partes conocidas sin demasiada coherencia.

## Intento 1.3: 50 épocas y 0.25
```
Yo, señor, aunque soy hijo de padres humildes, y hijos malas y partendose en camino y manera provechosa; y con favor que tuve de amigos y señores, y al mientro y tiene a entenria de mi casa y la tristeza y silencio de los moradores, tanto que nos acaeció estar dos o tres días y dornido en su costado por mi remedio un buen agujero. Esto es a las alguacil suelo, que yo puesto en pie con mi desdicha, despertando a este lacerado de mi amo y poniéndole más diligencia, dellos. Si deseo y por lo que toca a mi honra, aunque bien creo que será secreto, según lo poco que en este pueblo sobre el caño como de la conter. No es por más de una paja salido y descuidar, pensando que hallaría un buen asiento, mas no me ha sucedido como pensé.

"Señor, de mí -dije yo- ninguna pena tenga vuestra merced, que sé pasar una noche y aun más, si es verdad lo que yo digo y bien porque me sería a grandes voces, llamándome, procuró recordarme. Mas como me tocase con las manos, tentó lo mucha sanga Dios y aun más, porque a mí con amenazas me preguntaban, y 
```

### Observaciones
Mejor que el anterior en variedad, pero igual se nota que tira frases que ya aparecían en el texto. No hay una estructura clara.

## Intento 1.4.1: 100 épocas y 0.25
```
Yo, señor, aunque soy hijo de padres humildese que en las guardas nada de la suya diferenciaba. Fue luego a proballa, y con ella probó el maleficio y la negra que llaman honra lo que por vos no sufrirían!"

Ansí estaba yo a la puerta, mirando y con el canciento que le alcanzaron lo que él en un año no alcanzara: pienso que fueron doce o trece reales. Y él les dio lo noche y tan pía la mejor gustar el sabroso licor, sintió el desesperado ciego que agora tenía tiempo de tomar de mí venganza que el sobresaltado de mi amo lo oyó y creyó sin duda ser el silbo de la culebra; y cierto lo demosido, porque de hombre os habéis de convertir en malilla y si no. «Andá con Dios» os dicen. Y las más veces hacía del dormido, y en las mañas decíame él:

"Esta noche, mozo, ¿no sentiste nada? y había proción de sí llevaría har a aquel mezquino amo, mas por dos cosas lo dejaba: la primera, por no me atrever a mis piernas, por tener en las piernas de pura hambre. Vime claramente ir a la sepultura, si Dios y mi saber no había ninguna cosa de comer
```
## Intento 1.4.2: distinta start_string para probar
```
Cuando llegamos al mesón, a no había muerto, por quedar bien vezado de la hartura, tornando a mi cuotidiana hambre, más lo sentía algo que adobar.

"En mí teníades bien que hacer, y no haríades poco si me remediásedes", dijo mi amo y que encima del altar había, el cual habían traído para calentarse las manos porque hacía gran frío, me quería bien con el curado remedio, porque ya la caridad se subió al cielo, topóme Dios con un escudero que iba por la calle con la mano. Como tomase las rebanadas y mordiese en ellas pensando también llevar parte de la longaniza, y no pareciendo ellas pudiera negar la demanda. Pluguiera a Dios que lo hubiera hecho, y con mucha priesa fue a buscar lumbre. Y llega aquí a nuestra casa, y le damos de comer lo que podemos por amor de Dios, y a las noches se inventario, preguntándome qué tenía.

"Señores -dije yo-, lo que este mi amo tiene, según él me di con ellos parcial, y cuánto más hicieron los que, siéndoles contraria, con fuerza y maña remando, si no, verá ha y todo lo que e
```
### Observaciones
Este es el que sale mejor. Todavía hay cosas que parecen sacadas del texto, pero ya combina frases con más libertad y arma algo que suena un poco más fluido.

## Intento 1.5.1: 100 épocas y 0.3
```
Yo, señor, aunque soy hijo de padres humildese que amos tú esta hora entró una vieja que ensalmaba, y los vecinos, y comiénzanme a quitar trapos de la cabeza y viejo al del con el cabo de la capa sobre el lado izquierdo, sacó una llave de la manga y abrió su puerta a los que en ella entraban, aunque dentro della estaba un patio pequeño y razonables cámaras.

Desque tan recio como si diera con una gran calabaza, y cayó luego para atrás, así con ella, había muerto en la de los Gelves, y que ella confiaba en Dios no saldría peor hombre que desea mucho el provecho de las ánimas; mas pregunten a su merced si le pesa cuando le dicen al altoración y decir:

"¿Qué diremos a esto? ¡Nunca haber sentido ratones en esta casa sino agor de la Sagra de Toledo había predicado dos o tres días, haciendo sus acostumbradas diligencias, y decía:

"¿Qué tienes este había tenido a buscar clavos por la casa y por las paredes y tablillas a atapárselos. Venida la noche y su reposo, y también porque consideren los que heredaron nobles estados cuán po
```
## Intento 1.5.2:
```
Cuando llegamos al mesón, aunque decía se fiaban por un año, no aprovechaba y que estaban tan rebeldes en tomarla y que saba a los a parecía casa encantada. Estando así, díjome:

"Tú, mozo, ¿has comido?"

"No, señor -dije yo-, que eso no me dejaba de lo necesario. Digo verdad: si con mi serto que hábiera de manera que antes que el mal ciego sacase de mi boca su trompa, tal alteración sintió mi estado andaba murmurando de las bulas, diciendo como eran falsas y que el mesmo alguacil riñendo lo había de un agua el de mano y diome una gran calabazada en el diablo del toro, que más de tres días me duró el dolor de la boca, y dio muchas veces, por llevar a la posada con que él lo pasase, yo lo pasaba mal. Porque una mañana, levantos, con el cual probaré bien su suficiencia.

En un lacerada que todo fue acabado y la gente ida. Entonces salimos de la iglesia mayor, y yo tras él, y muy devotamente le vi oír misa y los otros oficios divinos, hacho tan pequeño tal ruindad!", y reían mucho el artificio, y decíanle: 
```
### Observaciones
Se nota mejor con la temperatura un poco elevada. Tiene menos frases tomadas directo del texto, se nota una buena estructura dialogo/descripción, y las oraciones tienen son gramáticamente correctas. Tiene algunos errores, no es tan coherente como el texto original y no suenan naturales muchas de las oraciones, pero es una generación fiel al estilo del texto original.